# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fcc34c47d90>
├── 'a' --> tensor([[-0.5307,  0.3382,  1.7672],
│                   [ 1.4367,  0.7224, -0.0908]])
└── 'x' --> <FastTreeValue 0x7fcc39c575e0>
    └── 'c' --> tensor([[ 0.2524,  1.1826, -0.8686,  1.1145],
                        [-0.4704, -1.8923, -1.0020,  1.2214],
                        [ 0.1898,  1.0298, -0.1952,  2.2338]])

In [4]:
t.a

tensor([[-0.5307,  0.3382,  1.7672],
        [ 1.4367,  0.7224, -0.0908]])

In [5]:
%timeit t.a

90.4 ns ± 0.766 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fcc34c47d90>
├── 'a' --> tensor([[-2.1113,  2.2173, -0.2739],
│                   [ 0.5778,  0.2426, -0.9716]])
└── 'x' --> <FastTreeValue 0x7fcc39c575e0>
    └── 'c' --> tensor([[ 0.2524,  1.1826, -0.8686,  1.1145],
                        [-0.4704, -1.8923, -1.0020,  1.2214],
                        [ 0.1898,  1.0298, -0.1952,  2.2338]])

In [7]:
%timeit t.a = new_value

102 ns ± 0.67 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5307,  0.3382,  1.7672],
               [ 1.4367,  0.7224, -0.0908]]),
    x: Batch(
           c: tensor([[ 0.2524,  1.1826, -0.8686,  1.1145],
                      [-0.4704, -1.8923, -1.0020,  1.2214],
                      [ 0.1898,  1.0298, -0.1952,  2.2338]]),
       ),
)

In [10]:
b.a

tensor([[-0.5307,  0.3382,  1.7672],
        [ 1.4367,  0.7224, -0.0908]])

In [11]:
%timeit b.a

75.1 ns ± 1.2 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6125, -0.9852, -0.2070],
               [ 0.2702, -0.2237,  0.3002]]),
    x: Batch(
           c: tensor([[ 0.2524,  1.1826, -0.8686,  1.1145],
                      [-0.4704, -1.8923, -1.0020,  1.2214],
                      [ 0.1898,  1.0298, -0.1952,  2.2338]]),
       ),
)

In [13]:
%timeit b.a = new_value

655 ns ± 4.46 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.13 µs ± 9.97 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.8 µs ± 120 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

275 µs ± 9.43 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

263 µs ± 7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fcc34c184f0>
├── 'a' --> tensor([[[-0.5307,  0.3382,  1.7672],
│                    [ 1.4367,  0.7224, -0.0908]],
│           
│                   [[-0.5307,  0.3382,  1.7672],
│                    [ 1.4367,  0.7224, -0.0908]],
│           
│                   [[-0.5307,  0.3382,  1.7672],
│                    [ 1.4367,  0.7224, -0.0908]],
│           
│                   [[-0.5307,  0.3382,  1.7672],
│                    [ 1.4367,  0.7224, -0.0908]],
│           
│                   [[-0.5307,  0.3382,  1.7672],
│                    [ 1.4367,  0.7224, -0.0908]],
│           
│                   [[-0.5307,  0.3382,  1.7672],
│                    [ 1.4367,  0.7224, -0.0908]],
│           
│                   [[-0.5307,  0.3382,  1.7672],
│                    [ 1.4367,  0.7224, -0.0908]],
│           
│                   [[-0.5307,  0.3382,  1.7672],
│                    [ 1.4367,  0.7224, -0.0908]]])
└── 'x' --> <FastTreeValue 0x7fcc34c382e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

44.6 µs ± 277 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fcb69de6070>
├── 'a' --> tensor([[-0.5307,  0.3382,  1.7672],
│                   [ 1.4367,  0.7224, -0.0908],
│                   [-0.5307,  0.3382,  1.7672],
│                   [ 1.4367,  0.7224, -0.0908],
│                   [-0.5307,  0.3382,  1.7672],
│                   [ 1.4367,  0.7224, -0.0908],
│                   [-0.5307,  0.3382,  1.7672],
│                   [ 1.4367,  0.7224, -0.0908],
│                   [-0.5307,  0.3382,  1.7672],
│                   [ 1.4367,  0.7224, -0.0908],
│                   [-0.5307,  0.3382,  1.7672],
│                   [ 1.4367,  0.7224, -0.0908],
│                   [-0.5307,  0.3382,  1.7672],
│                   [ 1.4367,  0.7224, -0.0908],
│                   [-0.5307,  0.3382,  1.7672],
│                   [ 1.4367,  0.7224, -0.0908]])
└── 'x' --> <FastTreeValue 0x7fcb69de60d0>
    └── 'c' --> tensor([[ 0.2524,  1.1826, -0.8686,  1.1145],
                        [-0.4704, -1.8923, -1.0020,  1.2214],
                 

In [23]:
%timeit t_cat(trees)

42.6 µs ± 630 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

105 µs ± 663 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2524,  1.1826, -0.8686,  1.1145],
                       [-0.4704, -1.8923, -1.0020,  1.2214],
                       [ 0.1898,  1.0298, -0.1952,  2.2338]],
              
                      [[ 0.2524,  1.1826, -0.8686,  1.1145],
                       [-0.4704, -1.8923, -1.0020,  1.2214],
                       [ 0.1898,  1.0298, -0.1952,  2.2338]],
              
                      [[ 0.2524,  1.1826, -0.8686,  1.1145],
                       [-0.4704, -1.8923, -1.0020,  1.2214],
                       [ 0.1898,  1.0298, -0.1952,  2.2338]],
              
                      [[ 0.2524,  1.1826, -0.8686,  1.1145],
                       [-0.4704, -1.8923, -1.0020,  1.2214],
                       [ 0.1898,  1.0298, -0.1952,  2.2338]],
              
                      [[ 0.2524,  1.1826, -0.8686,  1.1145],
                       [-0.4704, -1.8923, -1.0020,  1.2214],
                       [ 0.1898,  1.0298, -0.1952,  2.2338]],

In [26]:
%timeit Batch.stack(batches)

112 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2524,  1.1826, -0.8686,  1.1145],
                      [-0.4704, -1.8923, -1.0020,  1.2214],
                      [ 0.1898,  1.0298, -0.1952,  2.2338],
                      [ 0.2524,  1.1826, -0.8686,  1.1145],
                      [-0.4704, -1.8923, -1.0020,  1.2214],
                      [ 0.1898,  1.0298, -0.1952,  2.2338],
                      [ 0.2524,  1.1826, -0.8686,  1.1145],
                      [-0.4704, -1.8923, -1.0020,  1.2214],
                      [ 0.1898,  1.0298, -0.1952,  2.2338],
                      [ 0.2524,  1.1826, -0.8686,  1.1145],
                      [-0.4704, -1.8923, -1.0020,  1.2214],
                      [ 0.1898,  1.0298, -0.1952,  2.2338],
                      [ 0.2524,  1.1826, -0.8686,  1.1145],
                      [-0.4704, -1.8923, -1.0020,  1.2214],
                      [ 0.1898,  1.0298, -0.1952,  2.2338],
                      [ 0.2524,  1.1826, -0.8686,  1.1145],
                   

In [28]:
%timeit Batch.cat(batches)

250 µs ± 4.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

616 µs ± 37.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
